# Search and massive download
In this example, we will search for items using pygeodes, filter these items using geopandas dataframes and use a download queue to downloads these items and monitor the progress of the download.

## Imports
Let's start by importing geodes

In [1]:
from pygeodes import Geodes, Config

## Configuration
We configure using a config file located in our cwd

In [2]:
conf = Config.from_file("config.json")
geodes = Geodes(conf=conf)

## Searching products
We search for products in the *T31TCK* tile whose acquisition date is after *2023-01-01*

In [3]:
from pygeodes.utils.datetime_utils import complete_datetime_from_str

query = {
    "grid:code": {"eq": "T31TCK"},
    "end_datetime": {"gte": complete_datetime_from_str("2023-01-01")},
}
items, dataframe = geodes.search_items(query=query)

/work/scratch/data/fournih/test_env/lib/python3.11/site-packages/urllib3/connectionpool.py:1099: InsecureRequestWarning: Unverified HTTPS request is being made to host 'geodes-portal.cnes.fr'. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#tls-warnings
  warnings.warn(


Found 539 items matching your query, returning 80 as get_all parameter is set to False
80 item(s) found for query : {'grid:code': {'eq': 'T31TCK'}, 'end_datetime': {'gte': '2023-01-01T00:00:00.000000Z'}}



### Exploring results
We get a list ot items and a dataframe, we can work with the dataframe for instance : 

In [4]:
dataframe

,grid:code,end_datetime,id,collection,item,geometry
0,T31TCK,2024-05-02T10:56:21.024Z,URN:FEATURE:DATA:gdh:91c4d81a-0e4c-3dec-a9c2-4...,PEPS_S2_L1C,Item (URN:FEATURE:DATA:gdh:91c4d81a-0e4c-3dec-...,"POLYGON ((0.95006 45.13349, 0.45686 45.12551, ..."
1,T31TCK,2023-07-22T10:56:29.024Z,URN:FEATURE:DATA:gdh:e4e9708a-6005-3d57-9b36-0...,PEPS_S2_L1C,Item (URN:FEATURE:DATA:gdh:e4e9708a-6005-3d57-...,"POLYGON ((0.49965 44.138, 0.55032 44.1388, 0.5..."
2,T31TCK,2024-07-08T10:50:31.024Z,URN:FEATURE:DATA:gdh:48aeb103-69de-31b7-a3f3-d...,PEPS_S2_L1C,Item (URN:FEATURE:DATA:gdh:48aeb103-69de-31b7-...,"POLYGON ((0.45686 45.12551, 0.49965 44.138, 1...."
3,T31TCK,2023-04-08T10:56:21.024Z,URN:FEATURE:DATA:gdh:a6d9bb08-22ff-3be2-a715-9...,PEPS_S2_L1C,Item (URN:FEATURE:DATA:gdh:a6d9bb08-22ff-3be2-...,"POLYGON ((0.49965 44.138, 0.54163 44.13866, 0...."
4,T31TCK,2024-01-20T10:53:41.024Z,URN:FEATURE:DATA:gdh:084acd27-8d81-3dd1-adac-6...,PEPS_S2_L1C,Item (URN:FEATURE:DATA:gdh:084acd27-8d81-3dd1-...,"POLYGON ((0.45686 45.12551, 0.49965 44.138, 1...."
...,...,...,...,...,...,...
75,T31TCK,2023-07-19T10:46:29.024Z,URN:FEATURE:DATA:gdh:d38e0f8a-fa7b-352b-9c3f-b...,PEPS_S2_L1C,Item (URN:FEATURE:DATA:gdh:d38e0f8a-fa7b-352b-...,"POLYGON ((0.49965 44.138, 1.87192 44.1598, 1.8..."
76,T31TCK,2023-11-04T11:02:21.024Z,URN:FEATURE:DATA:gdh:444c28ff-2097-3fdf-b98e-7...,PEPS_S2_L1C,Item (URN:FEATURE:DATA:gdh:444c28ff-2097-3fdf-...,"POLYGON ((0.86769 44.92037, 0.90268 45.00649, ..."
77,T31TCK,2024-03-25T10:46:39.024Z,URN:FEATURE:DATA:gdh:7ad8089d-ac8d-33d9-b8c0-e...,PEPS_S2_L1C,Item (URN:FEATURE:DATA:gdh:7ad8089d-ac8d-33d9-...,"POLYGON ((0.45686 45.12551, 0.49965 44.138, 1...."
78,T31TCK,2023-06-04T10:46:21.024Z,URN:FEATURE:DATA:gdh:6236fb49-7523-30f7-8900-f...,PEPS_S2_L1C,Item (URN:FEATURE:DATA:gdh:6236fb49-7523-30f7-...,"POLYGON ((0.49965 44.138, 1.87192 44.1598, 1.8..."


### Adding columns
We want to filter on cloudcover, so we need to add the column to the dataframe.

In [5]:
items[0].list_available_keys()

{'area',
 'continent_code',
 'dataset',
 'datetime',
 'end_datetime',
 'endpoint_description',
 'endpoint_url',
 'eo:cloud_cover',
 'grid:code',
 'hydrology.rivers',
 'id',
 'identifier',
 'instrument',
 'keywords',
 'latest',
 'platform',
 'political.continents',
 'processing:level',
 'product:timeliness',
 'product:type',
 'proj:bbox',
 'references',
 's2:datatake_id',
 'sar:instrument_mode',
 'sat:absolute_orbit',
 'sat:orbit_state',
 'sat:relative_orbit',
 'sci:doi',
 'start_datetime',
 'version'}

We find we can use `spaceborne:cloudCover`, so we add it to the dataframe : 

In [6]:
from pygeodes.utils.formatting import format_items

dataframe_new = format_items(dataframe, {"eo:cloud_cover"})

### Filtering our results
Now that the cloud cover is in our dataframe, we can filter on it.

In [7]:
dataframe_filtered = dataframe_new[dataframe_new["eo:cloud_cover"] < 30]

In [8]:
dataframe_filtered

,grid:code,end_datetime,id,collection,item,geometry,eo:cloud_cover
1,T31TCK,2023-07-22T10:56:29.024Z,URN:FEATURE:DATA:gdh:e4e9708a-6005-3d57-9b36-0...,PEPS_S2_L1C,Item (URN:FEATURE:DATA:gdh:e4e9708a-6005-3d57-...,"POLYGON ((0.49965 44.138, 0.55032 44.1388, 0.5...",0.000000
2,T31TCK,2024-07-08T10:50:31.024Z,URN:FEATURE:DATA:gdh:48aeb103-69de-31b7-a3f3-d...,PEPS_S2_L1C,Item (URN:FEATURE:DATA:gdh:48aeb103-69de-31b7-...,"POLYGON ((0.45686 45.12551, 0.49965 44.138, 1....",0.000000
3,T31TCK,2023-04-08T10:56:21.024Z,URN:FEATURE:DATA:gdh:a6d9bb08-22ff-3be2-a715-9...,PEPS_S2_L1C,Item (URN:FEATURE:DATA:gdh:a6d9bb08-22ff-3be2-...,"POLYGON ((0.49965 44.138, 0.54163 44.13866, 0....",6.303467
4,T31TCK,2024-01-20T10:53:41.024Z,URN:FEATURE:DATA:gdh:084acd27-8d81-3dd1-adac-6...,PEPS_S2_L1C,Item (URN:FEATURE:DATA:gdh:084acd27-8d81-3dd1-...,"POLYGON ((0.45686 45.12551, 0.49965 44.138, 1....",0.553047
6,T31TCK,2024-09-19T11:08:39Z,URN:FEATURE:DATA:gdh:7c5a6f36-d16f-3640-b38a-8...,MUSCATE_SENTINEL2_SENTINEL2_L2A,Item (URN:FEATURE:DATA:gdh:7c5a6f36-d16f-3640-...,"POLYGON ((0.45686 45.12551, 1.85259 45.14807, ...",0.000000
7,T31TCK,2024-07-26T11:08:41Z,URN:FEATURE:DATA:gdh:2dc5d06f-cea7-3031-af0f-0...,MUSCATE_SENTINEL2_SENTINEL2_L2A,Item (URN:FEATURE:DATA:gdh:2dc5d06f-cea7-3031-...,"POLYGON ((0.45686 45.12551, 1.85259 45.14807, ...",8.000000
10,T31TCK,2024-04-02T11:08:37Z,URN:FEATURE:DATA:gdh:0c18cbce-4e42-3df1-a557-9...,MUSCATE_SENTINEL2_SENTINEL2_L2A,Item (URN:FEATURE:DATA:gdh:0c18cbce-4e42-3df1-...,"POLYGON ((0.45686 45.12551, 1.85259 45.14807, ...",6.000000
11,T31TCK,2024-08-05T11:08:41Z,URN:FEATURE:DATA:gdh:51739c13-41d7-328b-b4e5-1...,MUSCATE_SENTINEL2_SENTINEL2_L2A,Item (URN:FEATURE:DATA:gdh:51739c13-41d7-328b-...,"POLYGON ((0.45686 45.12551, 1.85259 45.14807, ...",0.000000
12,T31TCK,2023-11-26T10:58:40Z,URN:FEATURE:DATA:gdh:6e03e00e-c4f0-343e-9a3e-8...,MUSCATE_SENTINEL2_SENTINEL2_L2A,Item (URN:FEATURE:DATA:gdh:6e03e00e-c4f0-343e-...,"POLYGON ((0.45686 45.12551, 1.85259 45.14807, ...",0.000000
13,T31TCK,2023-06-04T10:58:53Z,URN:FEATURE:DATA:gdh:6a8cc460-c7c2-3cdf-9145-6...,MUSCATE_SENTINEL2_SENTINEL2_L2A,Item (URN:FEATURE:DATA:gdh:6a8cc460-c7c2-3cdf-...,"POLYGON ((0.45686 45.12551, 1.85259 45.14807, ...",28.000000


## Plotting

We can plot our results on a map :  

In [9]:
dataframe_filtered.explore()

## Downloading our items

We can download our results using the Profile system

In [10]:
from pygeodes.utils.profile import DownloadQueue, Profile

We reset our Profile to be sure to track only the downloads from the queue

In [11]:
Profile.reset()
items = dataframe_filtered["item"].values
queue = DownloadQueue(items)

In a **separate cell**, we run our queue

In [ ]:
queue.run()